# Scraper institutes

This notebook is a scraper used to retrieve generic information about detention centers in Italy (such as name, code, address, geographic coordinates). To do so, it uses the information stored at the webpage https://www.giustizia.it/giustizia/page/it/istituti_penitenziari. Because the information is loaded as javascript, we first use Selenium to save the code and then we use BeautifulSoup tand RegEx to parse the information.

In [1]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd
import re
import html
import datetime

# "Hey, open up a browser"
playwright = await async_playwright().start()
browser = await playwright.firefox.launch()
context = await browser.new_context(viewport={'width': 1280, 'height': 800})


# Create a new browser window
page = await context.new_page()

# Tell it to go to this page
await page.goto("https://www.giustizia.it/giustizia/page/it/istituti_penitenziari")

html_content = await page.content()

# Parsing the code with BeautifuSoup and RegEx
doc = BeautifulSoup(html_content)
# Find all institute marks
marks = re.findall(r"marker\.codiceIstituto.*?push", str(doc), re.DOTALL)
institutes_info = []
n = 0

for mark in marks:
    try:
        codice_istituto = re.search(r"marker\.codiceIstituto\s*=\s*\"(.*?)\";", mark).group(1)
        print(codice_istituto)
        title = re.search(r"marker\.title\s*=\s*\"(.*?)\";", mark).group(1)
        title = html.unescape(title)
        print(title)
        tipo = re.search(r"marker\.tipo\s*=\s*\"(.*?)\";", mark).group(1)
        lat = re.search(r"marker\.lat\s*=\s*(.*?);", mark).group(1)
        lon = re.search(r"marker\.lon\s*=\s*(.*?);", mark).group(1)
        provv_id = re.search(r"marker\.provv\s*=\s*{id:\s*(\d+),", mark).group(1)
        provv_name = re.search(r"marker\.provv\s*=\s*{id:\s*\d+,\s*name:\s*\"(.*?)\"}", mark).group(1)
        indirizzo = re.search(r"marker\.indirizzo\s*=\s*\"(.*?)\";", mark).group(1)
        telefono = re.search(r"marker\.telefono\s*=\s*\"(.*?)\";", mark).group(1)
        mii = re.search(r"marker\.mii\s*=\s*\'(.*?)\'", mark).group(1)


        institute_info = {
            "id_istituto": mii,
            "codice_istituto" : codice_istituto,
            "nome_istituto" : title,
            "tipo_istituto" : tipo,
            "latitudine" : lat,
            "longitude" : lon,
            "id_provveditorato" : provv_id,
            "nome_provveditorato" : provv_name,
            "indirizzo" : indirizzo,
            "telefono" : telefono
        }

        institutes_info.append(institute_info)
        print(f"mark scraped number {n}")
        n += 1
        print("####")

    except AttributeError:
        continue

df = pd.DataFrame(institutes_info)

HH44
Reggio Calabria Arghillà
mark scraped number 0
####
BB43
Brescia Verziano
mark scraped number 1
####
BB16
Busto Arsizio
mark scraped number 2
####
BB17
Como
mark scraped number 3
####
BB18
Cremona
mark scraped number 4
####
BB19
Lecco
mark scraped number 5
####
BB20
Lodi
mark scraped number 6
####
BB21
Mantova
mark scraped number 7
####
BB22
Monza
mark scraped number 8
####
BB23
Pavia
mark scraped number 9
####
BB24
Sondrio
mark scraped number 10
####
BB25
Varese
mark scraped number 11
####
BB37
Voghera
mark scraped number 12
####
HH11
Catanzaro
mark scraped number 13
####
HH39
Crotone
mark scraped number 14
####
HH19
Palmi
mark scraped number 15
####
HH20
Paola
mark scraped number 16
####
HH41
Rossano
mark scraped number 17
####
HH07
Vibo Valentia
mark scraped number 18
####
FF30
Ariano Irpino
mark scraped number 19
####
FF03
Aversa
mark scraped number 20
####
FF29
Carinola
mark scraped number 21
####
FF64
Eboli
mark scraped number 22
####
CC03
Bolzano
mark scraped number 23
####

In [2]:
df.head()

,id_istituto,codice_istituto,nome_istituto,tipo_istituto,latitudine,longitude,id_provveditorato,nome_provveditorato,indirizzo,telefono
0,MII179988,HH44,Reggio Calabria Arghillà,Casa circondariale,38.1929515,15.681926617,8,Calabria,Contrada Rugola n. snc - 89135 Reggio Calabria,+39 0965 670031
1,MII172610,BB43,Brescia Verziano,Casa di reclusione,45.4996058,10.18776766,1,Lombardia,Via Flero n.157 - 25124 Brescia,+39 030 3580386
2,MII172320,BB16,Busto Arsizio,Casa circondariale,45.6119057,8.8520522,1,Lombardia,Via per Cassano Magnago n.102 - 21052 Busto Ar...,+39 0331 685777
3,MII173712,BB17,Como,Casa circondariale,45.9394759,9.14941015,1,Lombardia,Via Bassone n.11 - 22100 Como,+39 031 590848
4,MII173747,BB18,Cremona,Casa circondariale,45.1334796,10.0248716,1,Lombardia,Via Palosca n.2 - 26100 Cremona,+39 0372 450505


In [3]:
await page.close()

## Some basic cleaning

In [4]:
df['tipo_istituto'].value_counts()

tipo_istituto
Casa circondariale                           90
Casa circondariale -                         43
Casa di reclusione                           29
Casa di reclusione -                         19
Casa di reclusione femminile                  1
Casa di lavoro                                1
Casa circondariale femminile                  1
Casa circondariale - Alfredo Paragano         1
Casa circondariale - Michele Gaglione         1
Casa circondariale  N.C.  - Nicandro Izzo     1
Casa di reclusione - Gennaro De Angelis       1
Istituto custodia attenuata per madri         1
Istituti di pena -                            1
Name: count, dtype: int64

In [5]:
# Cleaning institute names
df['tipo_istituto'] = df['tipo_istituto'].str.replace("-","").str.strip()

In [6]:
df['tipo_istituto'].value_counts()

tipo_istituto
Casa circondariale                          133
Casa di reclusione                           48
Casa di reclusione femminile                  1
Casa di lavoro                                1
Casa circondariale femminile                  1
Casa circondariale  Alfredo Paragano          1
Casa circondariale  Michele Gaglione          1
Casa circondariale  N.C.   Nicandro Izzo      1
Casa di reclusione  Gennaro De Angelis        1
Istituto custodia attenuata per madri         1
Istituti di pena                              1
Name: count, dtype: int64

In [7]:
current_day = datetime.datetime.now().strftime("%Y-%m-%d")

In [8]:
df.to_csv(f'../outputs/clean/istituti_info.csv', index=False)